# Chat Agent Debug Notebook

This notebook allows you to test the complete chat agent flow with real queries and see the raw objects at each step.

In [1]:
# Setup and imports
import os
import sys
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from pprint import pprint
import json

# Import our chat agent components
from chat_parser import ChatParser
from query_builder import QueryBuilder
from response_formatter import ResponseFormatter
from agent_tools import initialize_tools
import agent_tools

print("✅ All imports successful!")

✅ All imports successful!


In [2]:
# Initialize Elasticsearch connection
load_dotenv(dotenv_path=".env", override=True)

# Initialize Elasticsearch client (6.x version)
es = Elasticsearch(
    hosts=[os.getenv('ES_HOST')],
    http_auth=(os.getenv('ES_USER'), os.getenv('ES_PASS')),
    verify_certs=False  # Set to True in production
)

print(f"✅ Connected to ES: {es.ping()}")
print(f"ES Version: {es.info()['version']['number']}")

# Initialize the agent tools
initialize_tools(es)
print("✅ Agent tools initialized!")

/Users/filipberntsson/Dev/es_workspace/venv/lib/python3.13/site-packages/elasticsearch/connection/http_urllib3.py:176: UserWarning: Connecting to elk-test.cthb.se using SSL with verify_certs=False is insecure.
  warnings.warn(


✅ Connected to ES: True
ES Version: 6.8.23
✅ Agent tools initialized!


In [3]:
# Initialize chat agent components
parser = ChatParser()
builder = QueryBuilder(agent_tools)
formatter = ResponseFormatter()

print("✅ Chat agent components initialized!")
print(f"Parser: {parser}")
print(f"Builder: {builder}")
print(f"Formatter: {formatter}")

✅ Chat agent components initialized!
Parser: <chat_parser.ChatParser object at 0x11c8d7a10>
Builder: <query_builder.QueryBuilder object at 0x11cce0ec0>
Formatter: <response_formatter.ResponseFormatter object at 0x11cce1010>


## Debug Function

This function will show you the complete flow with detailed debugging information.

In [ ]:
def debug_chat_agent(user_query, show_raw=True, include_suggestions=False):
    """
    Debug the complete chat agent flow.
    
    Args:
        user_query: The natural language query
        show_raw: Whether to show raw objects
        include_suggestions: Whether to include suggestions in response
    """
    print("=" * 80)
    print(f"🔍 DEBUGGING QUERY: '{user_query}'")
    print("=" * 80)
    
    try:
        # Step 1: Parse the query
        print("\n📝 STEP 1: PARSING QUERY")
        print("-" * 40)
        parsed_query = parser.parse(user_query)
        
        print(f"Intent: {parsed_query.intent.value}")
        print(f"Entity Type: {parsed_query.entity_type}")
        print(f"Confidence: {parsed_query.confidence:.2f}")
        
        if parsed_query.author_name:
            print(f"Author: {parsed_query.author_name}")
        if parsed_query.journal_name:
            print(f"Journal: {parsed_query.journal_name}")
        if parsed_query.search_terms:
            print(f"Search Terms: {parsed_query.search_terms}")
        if parsed_query.limit:
            print(f"Limit: {parsed_query.limit}")
        if parsed_query.group_by:
            print(f"Group By: {parsed_query.group_by}")
        if parsed_query.filters:
            print(f"Filters: {parsed_query.filters}")
        
        if show_raw:
            print("\n📋 RAW PARSED QUERY:")
            pprint(vars(parsed_query))
        
        # Step 2: Build the query
        print("\n🔧 STEP 2: BUILDING QUERY")
        print("-" * 40)
        query_spec = builder.build_query(parsed_query)
        
        print(f"Function: {query_spec['function']}")
        print(f"Post-process: {query_spec['post_process']}")
        print(f"Arguments: {query_spec['args']}")
        if 'post_process_args' in query_spec:
            print(f"Post-process Args: {query_spec['post_process_args']}")
        
        # Show strategy information for author queries
        if query_spec['function'] == 'search_by_author' and 'strategy' in query_spec['args']:
            strategy = query_spec['args']['strategy']
            print(f"🎯 Author Search Strategy: {strategy}")
            
            # Explain what this strategy means
            strategy_explanations = {
                'exact': 'Uses match_phrase for precise full name matching',
                'partial': 'Uses match for broader surname matching',
                'fuzzy': 'Uses fuzzy matching for typos/variations'
            }
            if strategy in strategy_explanations:
                print(f"   → {strategy_explanations[strategy]}")
        
        if show_raw:
            print("\n📋 RAW QUERY SPEC:")
            pprint(query_spec)
        
        # Step 3: Execute the query
        print("\n⚡ STEP 3: EXECUTING QUERY")
        print("-" * 40)
        query_result = builder.execute_query(query_spec)
        
        print(f"Result Type: {query_result['type']}")
        if query_result['type'] == 'count':
            print(f"Count: {query_result['count']}")
            print(f"Query: {query_result['query']}")
        elif query_result['type'] == 'list':
            print(f"Total Results: {query_result['total']}")
            print(f"Results Returned: {len(query_result['results'])}")
            print(f"Session ID: {query_result['session_id']}")
        elif query_result['type'] == 'keyword_stats':
            print(f"Keywords Found: {len(query_result['keywords'])}")
            print(f"Session ID: {query_result['session_id']}")
            if query_result['keywords']:
                print(f"Top Keyword: {query_result['keywords'][0]['value']} ({query_result['keywords'][0]['count']} occurrences)")
        
        if show_raw:
            print("\n📋 RAW QUERY RESULT:")
            pprint(query_result)
        
        # Step 4: Format the response
        print("\n🎨 STEP 4: FORMATTING RESPONSE")
        print("-" * 40)
        formatted_response = formatter.format_response(
            query_result, 
            include_suggestions=include_suggestions
        )
        
        print(f"Response Type: {formatted_response['type']}")
        if formatted_response['type'] == 'text':
            print(f"Content: {formatted_response['content']}")
        elif formatted_response['type'] == 'list':
            print(f"Items: {len(formatted_response['items'])}")
            print(f"Total: {formatted_response['total']}")
            if formatted_response['items']:
                print(f"First Item: {formatted_response['items'][0]['title']}")
        elif formatted_response['type'] == 'table':
            print(f"Headers: {formatted_response['headers']}")
            print(f"Rows: {len(formatted_response['rows'])}")
            print(f"Summary: {formatted_response['summary']}")
        
        if include_suggestions and 'suggestions' in formatted_response:
            print(f"Suggestions: {formatted_response['suggestions']}")
        
        if show_raw:
            print("\n📋 RAW FORMATTED RESPONSE:")
            pprint(formatted_response)
        
        print("\n" + "=" * 80)
        print("✅ QUERY PROCESSING COMPLETE!")
        print("=" * 80)
        
        return {
            'parsed_query': parsed_query,
            'query_spec': query_spec,
            'query_result': query_result,
            'formatted_response': formatted_response
        }
        
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        print(f"Error type: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        return None

## Test Your Queries Here!

Use the cell below to test different queries. Change the `user_query` variable to test different types of queries.

In [10]:
# 🔍 TEST YOUR QUERY HERE!
# Change this to test different queries
user_query = "List all articles that Martin Nilson Jacobi has published"

# Run the debug function
result = debug_chat_agent(
    user_query=user_query,
    show_raw=True,  # Set to False to hide raw objects
    include_suggestions=True  # Set to False to hide suggestions
)

🔍 DEBUGGING QUERY: 'List all articles that Martin Nilson Jacobi has published'

📝 STEP 1: PARSING QUERY
----------------------------------------
Intent: list
Entity Type: author
Confidence: 1.00
Author: Nilson Jacobi
Journal: Nilson Jacobi has published

📋 RAW PARSED QUERY:
{'author_name': 'Nilson Jacobi',
 'confidence': 0.9999999999999999,
 'entity_type': 'author',
 'filters': {},
 'group_by': None,
 'intent': <QueryIntent.LIST: 'list'>,
 'journal_name': 'Nilson Jacobi has published',
 'limit': None,
 'search_terms': None}

🔧 STEP 2: BUILDING QUERY
----------------------------------------
Function: search_by_author
Post-process: list
Arguments: {'author_name': 'Nilson Jacobi'}

📋 RAW QUERY SPEC:
{'args': {'author_name': 'Nilson Jacobi'},
 'function': 'search_by_author',
 'post_process': 'list'}

⚡ STEP 3: EXECUTING QUERY
----------------------------------------
Result Type: list
Total Results: 56
Results Returned: 5
Session ID: a2dc5614-b69c-4cc3-9c05-05fc96967fae

📋 RAW QUERY RESULT:

## Quick Test Examples

Here are some example queries you can test. Just uncomment one and run the cell above.

In [7]:
# Example queries to test:
example_queries = [
    "How many articles has Christian Fager published?",
    "List all articles that Anna Dubois has published",
    "Find papers about machine learning from 2023",
    "What are the top 10 keywords on publications per year from 2020 to 2024?",
    "How many publications have been published in Nature in 2023?",
    "Show me recent papers by Erik Lind from 2022 to 2024",
    "Tell me about quantum computing",
    "List publications about artificial intelligence"
]

print("Example queries you can test:")
for i, query in enumerate(example_queries, 1):
    print(f"{i}. {query}")
    
print("\nTo test a query, copy it to the cell above and run it!")

Example queries you can test:
1. How many articles has Christian Fager published?
2. List all articles that Anna Dubois has published
3. Find papers about machine learning from 2023
4. What are the top 10 keywords on publications per year from 2020 to 2024?
5. How many publications have been published in Nature in 2023?
6. Show me recent papers by Erik Lind from 2022 to 2024
7. Tell me about quantum computing
8. List publications about artificial intelligence

To test a query, copy it to the cell above and run it!


## Quick Test Runner

Run multiple queries quickly to see how they're parsed.

In [11]:
# Quick test multiple queries
test_queries = [
    "How many papers has Christian Fager published?",
    "List Anna Dubois publications",
    "Find quantum papers"
]

for query in test_queries:
    print(f"\n🔍 Testing: '{query}'")
    parsed = parser.parse(query)
    print(f"   Intent: {parsed.intent.value}, Entity: {parsed.entity_type}, Confidence: {parsed.confidence:.2f}")
    if parsed.author_name:
        print(f"   Author: {parsed.author_name}")
    if parsed.search_terms:
        print(f"   Search: {parsed.search_terms}")
    if parsed.filters:
        print(f"   Filters: {parsed.filters}")


🔍 Testing: 'How many papers has Christian Fager published?'
   Intent: count, Entity: author, Confidence: 0.90
   Author: Christian Fager
   Search: has christian fager published?

🔍 Testing: 'List Anna Dubois publications'
   Intent: list, Entity: publication, Confidence: 0.70

🔍 Testing: 'Find quantum papers'
   Intent: search, Entity: publication, Confidence: 0.70
   Search: quantum papers


## Raw ES Tools Testing

Test the underlying ES tools directly.

In [14]:
# Test raw ES tools
print("Testing raw ES tools...")

# Test search_by_author a
print("\n1. Testing search_by_author a:")
try:
    result = agent_tools.search_by_author("Fager")
    print(f"   Found {result['total_results']} publications")
    print(f"   Session ID: {result['session_id']}")
    if result['sample_results']:
        print(f"   First result: {result['sample_results'][0]['title']}")
except Exception as e:
    print(f"   Error: {e}")

# Test search_by_author b
print("\n1. Testing search_by_author b:")
try:
    result = agent_tools.search_by_author("Christian Fager")
    print(f"   Found {result['total_results']} publications")
    print(f"   Session ID: {result['session_id']}")
    if result['sample_results']:
        print(f"   First result: {result['sample_results'][0]['title']}")
except Exception as e:
    print(f"   Error: {e}")

# Test search_publications
print("\n2. Testing search_publications:")
try:
    result = agent_tools.search_publications(query="machine learning", size=5)
    print(f"   Found {result['total_results']} publications")
    print(f"   Session ID: {result['session_id']}")
    if result['sample_results']:
        print(f"   First result: {result['sample_results'][0]['title']}")
except Exception as e:
    print(f"   Error: {e}")

# Test statistics
print("\n3. Testing get_statistics_summary:")
try:
    stats = agent_tools.get_statistics_summary()
    print(f"   Total publications: {stats['total_publications']:,}")
    print(f"   Recent years: {stats['years'][:3]}")
    print(f"   Top publication types: {stats['publication_types'][:3]}")
except Exception as e:
    print(f"   Error: {e}")

Testing raw ES tools...

1. Testing search_by_author a:
   Found 304 publications
   Session ID: 6fc807b4-071f-4084-97a5-422fbe49604e
   First result: Characterisation of an FMCW radar transceiver

1. Testing search_by_author b:
   Found 2679 publications
   Session ID: a09ebce4-b4d5-4373-b734-c670f3c9921d
   First result: Characterisation of an FMCW radar transceiver

2. Testing search_publications:
   Found 4628 publications
   Session ID: 81ed0f7d-e3b5-4339-9a61-ad795e4076c4
   First result: Data Integration Using Machine Learning

3. Testing get_statistics_summary:
   Total publications: 95,697
   Recent years: [{'value': 2025, 'count': 1625}, {'value': 2024, 'count': 3962}, {'value': 2023, 'count': 3937}]
   Top publication types: [{'value': 'Journal article', 'count': 42519}, {'value': 'Paper in proceeding', 'count': 23473}, {'value': 'Doctoral thesis', 'count': 5219}]


## Component Testing

Test individual components in isolation.

In [ ]:
# Test parser with different query types
test_cases = [
    "How many articles has Christian Fager published?",
    "List all papers by Anna Dubois from 2020 to 2023",
    "What are the top 5 keywords per year from 2020 to 2024?",
    "Find papers about quantum computing",
    "Show recent publications in Nature",
    "Tell me about machine learning",
    "Random nonsense query"
]

print("Testing parser with different query types:")
print("=" * 60)

for query in test_cases:
    print(f"\nQuery: '{query}'")
    parsed = parser.parse(query)
    print(f"  Intent: {parsed.intent.value}")
    print(f"  Entity: {parsed.entity_type}")
    print(f"  Confidence: {parsed.confidence:.2f}")
    
    if parsed.author_name:
        print(f"  Author: {parsed.author_name}")
    if parsed.search_terms:
        print(f"  Search: {parsed.search_terms}")
    if parsed.limit:
        print(f"  Limit: {parsed.limit}")
    if parsed.filters:
        print(f"  Filters: {parsed.filters}")
    
    print("-" * 40)

In [ ]:
# 📊 COMPREHENSIVE STRATEGY TESTING
# Test our strategy-based search with real author names from the database

print("=" * 80)
print("COMPREHENSIVE STRATEGY-BASED AUTHOR SEARCH TESTING")
print("=" * 80)

# Test cases with real authors from the database
test_cases = [
    {
        "full_name": "Christian Fager",
        "surname": "Fager",
        "description": "Known high-impact researcher"
    },
    {
        "full_name": "Anna Dubois", 
        "surname": "Dubois",
        "description": "Common name pattern"
    },
    {
        "full_name": "Erik Lind",
        "surname": "Lind", 
        "description": "Short common surname"
    }
]

results_summary = []

for i, test_case in enumerate(test_cases, 1):
    print(f"\n{i}. TESTING: {test_case['description']}")
    print("-" * 60)
    
    # Test full name (should use exact strategy)
    print(f"   Full name query: 'How many papers has {test_case['full_name']} published?'")
    try:
        parsed_full = parser.parse(f"How many papers has {test_case['full_name']} published?")
        query_spec_full = builder.build_query(parsed_full)
        result_full = builder.execute_query(query_spec_full)
        
        full_strategy = query_spec_full['args']['strategy']
        full_count = result_full['count']
        
        print(f"   → Strategy: {full_strategy}")
        print(f"   → Results: {full_count}")
        
    except Exception as e:
        print(f"   → Error: {e}")
        full_strategy = "error"
        full_count = 0
    
    # Test surname only (should use partial strategy)
    print(f"   Surname query: 'How many papers has {test_case['surname']} published?'")
    try:
        parsed_surname = parser.parse(f"How many papers has {test_case['surname']} published?")
        query_spec_surname = builder.build_query(parsed_surname)
        result_surname = builder.execute_query(query_spec_surname)
        
        surname_strategy = query_spec_surname['args']['strategy']
        surname_count = result_surname['count']
        
        print(f"   → Strategy: {surname_strategy}")
        print(f"   → Results: {surname_count}")
        
    except Exception as e:
        print(f"   → Error: {e}")
        surname_strategy = "error"
        surname_count = 0
    
    # Calculate difference
    if full_count > 0 and surname_count > 0:
        diff = surname_count - full_count
        ratio = surname_count / full_count
        print(f"   → Difference: {diff} more results for surname ({ratio:.1f}x)")
        print(f"   → This demonstrates proper strategy selection!")
    else:
        diff = 0
        ratio = 0
        print(f"   → Could not calculate difference")
    
    results_summary.append({
        'author': test_case['full_name'],
        'full_name_strategy': full_strategy,
        'full_name_count': full_count,
        'surname_strategy': surname_strategy,
        'surname_count': surname_count,
        'difference': diff,
        'ratio': ratio
    })

print("\n" + "=" * 80)
print("RESULTS SUMMARY TABLE")
print("=" * 80)
print(f"{'Author':<20} {'Full Strategy':<12} {'Full Count':<10} {'Surname Strategy':<15} {'Surname Count':<12} {'Difference':<10}")
print("-" * 80)

for result in results_summary:
    print(f"{result['author']:<20} {result['full_name_strategy']:<12} {result['full_name_count']:<10} {result['surname_strategy']:<15} {result['surname_count']:<12} {result['difference']:<10}")

print("\n" + "=" * 80)
print("CONCLUSION")
print("=" * 80)
print("✅ Strategy-based search successfully implemented!")
print("✅ Full names use 'exact' strategy (match_phrase)")
print("✅ Surnames use 'partial' strategy (match)")
print("✅ Surname searches return MORE results than full name searches")
print("✅ This solves the BM25 scoring problem identified earlier!")
print("=" * 80)

## Strategy-Based Search Results Documentation

This section documents the effectiveness of our strategy-based author search implementation that solves the BM25 scoring problem.

In [ ]:
# 🔬 STRATEGY TESTING SECTION
# Let's test the different strategies with real author names

print("Testing different author search strategies:")
print("=" * 60)

# Test 1: Full name - should use "exact" strategy
print("\n1. Testing FULL NAME (should use 'exact' strategy):")
result1 = debug_chat_agent("How many papers has Christian Fager published?", show_raw=False)

# Test 2: Single surname - should use "partial" strategy  
print("\n2. Testing SINGLE SURNAME (should use 'partial' strategy):")
result2 = debug_chat_agent("How many papers has Fager published?", show_raw=False)

# Test 3: Compare results
print("\n3. STRATEGY COMPARISON:")
if result1 and result2:
    full_name_count = result1['query_result']['count']
    surname_count = result2['query_result']['count']
    print(f"   Full name 'Christian Fager': {full_name_count} results")
    print(f"   Surname 'Fager': {surname_count} results")
    print(f"   Difference: {surname_count - full_name_count} more results for surname")
    print(f"   This demonstrates the BM25 issue is solved with strategy-based queries!")
else:
    print("   Error in one of the queries")

print("\n" + "=" * 60)